In [ ]:
from PIL import Image, ImageDraw, ImageFont
import textwrap
from dataclasses import dataclass


@dataclass
class Event:
    name: str
    date: str
    time: str
    location: str


class EventImageGenerator:
    def __init__(self, event):
        self.event = event
        self.width = self.height = 1080
        self.border_thickness = 15
        self.border_radius = 30

    def generate(self):
        # Create a base image with the border
        base = Image.new(
            "RGB",
            (
                self.width + 2 * self.border_thickness,
                self.height + 2 * self.border_thickness,
            ),
            (0, 92, 144),
        )  # Blue color
        draw = ImageDraw.Draw(base)

        # Draw the black border with rounded corners
        border_color = (0, 0, 0)  # Black border color
        draw.rounded_rectangle(
            [
                (self.border_thickness, self.border_thickness),
                (
                    self.width + self.border_thickness,
                    self.height + self.border_thickness,
                ),
            ],
            radius=self.border_radius,
            outline=border_color,
            width=self.border_thickness,
        )

        # Load fonts
        try:
            # Regular text font
            font_event = ImageFont.truetype(
                "/usr/share/fonts/truetype/noto/NotoSans-Bold.ttf", 50
            )
            font_details = ImageFont.truetype(
                "/usr/share/fonts/truetype/noto/NotoSans-Regular.ttf", 45
            )
        except OSError:
            print("Font not found, using default font.")
            font_event = ImageFont.load_default()
            font_details = font_event

        # Load icon images
        try:
            calendar_icon = Image.open("icons/calendar.png")
            clock_icon = Image.open("icons/clock.png")
            map_pin_icon = Image.open("icons/map-pin.png")
        except FileNotFoundError:
            print("Error: One or more icons not found in the 'icons' directory.")
            return

        # Set some padding for the text
        padding = 50
        max_width = self.width - 2 * padding

        # Function to wrap text into multiple lines if it exceeds the width
        def wrap_text(text, font, max_width):
            wrapped_lines = textwrap.wrap(
                text, width=40
            )  # Wrap text after a certain number of characters
            return wrapped_lines

        # Function to draw icons and corresponding text with proper alignment
        def draw_icon_and_text(icon, text, y_position, font, is_first_line=False):
            if icon:
                # Resize the icon to fit with the text
                icon_resized = icon.resize((40, 40))

                # Get the bounding box of the text to calculate its height
                text_bbox = draw.textbbox((padding + 50, y_position), text, font=font)
                text_height = text_bbox[3] - text_bbox[1]  # Height of the text box

                # Calculate the total height that will be occupied by the icon and the text
                total_height = max(icon_resized.height, text_height)

                # Calculate y-position adjustments so both the icon and text are vertically centered
                icon_offset = (
                    total_height - icon_resized.height
                ) // 2  # Center the icon vertically
                text_offset = (
                    total_height - text_height
                ) // 2  # Center the text vertically

                # Lower the icon position by a fixed amount (if necessary)
                icon_offset += 10  # Move the icon down slightly for better alignment

                # Place icon and text on the image (only draw the icon if it's the first line)
                if is_first_line:
                    base.paste(
                        icon_resized, (padding, y_position + icon_offset), icon_resized
                    )  # Place icon
                draw.text(
                    (padding + (50 if is_first_line else 0), y_position + text_offset),
                    text,
                    font=font,
                    fill="white",
                )  # Place text

                # Return the new y-position after drawing the icon and text
                return (
                    y_position + total_height + 20
                )  # Add some space for the next section
            else:
                # Just draw text if there's no icon
                text_bbox = draw.textbbox((padding + 50, y_position), text, font=font)
                text_height = text_bbox[3] - text_bbox[1]
                text_offset = text_height // 2

                # Place the text on the image
                draw.text(
                    (padding + 50, y_position + text_offset),
                    text,
                    font=font,
                    fill="white",
                )

                # Return the new y-position after drawing the text
                return y_position + text_height + 20

        # Wrap and draw the event name
        event_lines = wrap_text(self.event.name, font_event, max_width)
        y_position = padding

        for line in event_lines:
            y_position += 70  # Add some vertical space between lines

        # Draw the event date with calendar icon (only on the first line)
        y_position = draw_icon_and_text(
            calendar_icon, self.event.date, y_position, font_details, is_first_line=True
        )

        # Draw the event time with clock icon (only on the first line)
        y_position = draw_icon_and_text(
            clock_icon, self.event.time, y_position, font_details, is_first_line=True
        )

        # Wrap and draw the event location (fix the wrapping for longer location)
        location_lines = wrap_text(self.event.location, font_details, max_width)

        # Adjust the spacing between the first and second lines of location with this variable
        space_between_first_and_second_line = -20  # This is the value to adjust only the space between the first and second location lines

        for i, line in enumerate(location_lines):
            # For location, only show the map pin icon on the first line
            if i == 0:
                # Draw the map pin icon and the first line of location
                y_position = draw_icon_and_text(
                    map_pin_icon, line, y_position, font_details, is_first_line=True
                )
            else:
                # For the second line and beyond, don't show the map pin and add reduced space only between the first and second line
                if i == 1:
                    y_position = draw_icon_and_text(
                        None,
                        line,
                        y_position + space_between_first_and_second_line,
                        font_details,
                        is_first_line=False,
                    )
                else:
                    y_position = draw_icon_and_text(
                        None, line, y_position, font_details, is_first_line=False
                    )

        # Calculate total content height to center the text
        total_content_height = y_position - padding  # The total height of the content

        # Calculate the starting Y position to center the content vertically
        y_position_offset = (
            self.height - total_content_height
        ) // 2 - 70  # Fine-tuned the offset value by -15

        # Create the final image with the vertical offset applied
        base = Image.new(
            "RGB",
            (
                self.width + 2 * self.border_thickness,
                self.height + 2 * self.border_thickness,
            ),
            (0, 92, 144),
        )  # Blue color
        draw = ImageDraw.Draw(base)

        # Redraw the border
        draw.rounded_rectangle(
            [
                (self.border_thickness, self.border_thickness),
                (
                    self.width + self.border_thickness,
                    self.height + self.border_thickness,
                ),
            ],
            radius=self.border_radius,
            outline=border_color,
            width=self.border_thickness,
        )

        # Re-initialize y_position with the vertical offset applied
        y_position = (
            padding + y_position_offset
        )  # Apply the offset for vertical centering

        # Redraw event name
        for line in event_lines:
            draw.text((padding, y_position), line, font=font_event, fill="white")
            y_position += 70  # Add some vertical space between lines

        # Redraw event date and time with icons
        y_position = draw_icon_and_text(
            calendar_icon, self.event.date, y_position, font_details, is_first_line=True
        )
        y_position = draw_icon_and_text(
            clock_icon, self.event.time, y_position, font_details, is_first_line=True
        )

        # Redraw the event location
        for i, line in enumerate(location_lines):
            if i == 0:
                y_position = draw_icon_and_text(
                    map_pin_icon, line, y_position, font_details, is_first_line=True
                )
            else:
                if i == 1:
                    y_position = draw_icon_and_text(
                        None,
                        line,
                        y_position + space_between_first_and_second_line,
                        font_details,
                        is_first_line=False,
                    )
                else:
                    y_position = draw_icon_and_text(
                        None, line, y_position, font_details, is_first_line=False
                    )

        # Save the image
        base.save("event_image.png")

        # Show the image
        base.show()


# Sample event details
event = Event(
    name="Chicago (DePaul): Stop the Hate: Rally for Jewish Students",
    date="Thursday, Nov 21",
    time="5:00 pm",
    location=(
        "DePaul University - Lincoln Park Student Center, 2250 N. Sheffield Ave."
    ),
)

EventImageGenerator(event).generate()


kf.service.services: KApplicationTrader: mimeType "x-scheme-handler/file" not found
kf.i18n.kuit: "Unknown subcue ':whatsthis,' in UI marker in context {@info:whatsthis, %1 the action's text}."
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-aptus-mos"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-arq"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-bay"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-bmq"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-cap"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-cine"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-cs1"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-dc2"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-drf"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-dxo"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-epson-eip"
org.kde.kdegraphics.gwe